In [11]:
!pip install -q keras-ocr
!pip install pillow

Defaulting to user installation because normal site-packages is not writeable


In [12]:
import keras_ocr
import matplotlib.pyplot as plt
import os
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
model = keras_ocr.pipeline.Pipeline()

In [14]:
import tensorflow
import keras


In [19]:
image_dir = "C:\\Users\\Midhunsai Pondhuru\\Downloads\\DATA_SET"


In [20]:
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]


In [21]:
for image_file in image_files:
    image = keras_ocr.tools.read(image_file)
    prediction = model.recognize([image])
    font_label = conv_label(np.argmax(model.predict(image)))
    plt.figure(figsize=(10, 20))
    plt.imshow(image)
    plt.title(f'Predicted Font: {font_label}')
    plt.show()

In [22]:
def conv_label(label):
    font_dict = {
        0: 'Georgia',
        1: 'Gill Sans',
        2: 'Helvetica',
        3: 'Hombre',
        4: 'Lato',
        5: 'LCD Mono',
        6: 'Lucida Bright',
        7: 'Minion',
        8: 'Monotype Corsiva',
        9: 'Mrs Eaves',
        10: 'Myriad',
        11: 'Times New Roman',
        12: 'Verdana'
    }
    return font_dict.get(label)


In [24]:
import os
import random
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [25]:
random.seed(42)

In [26]:
data_dir = "C:\\Users\\Midhunsai Pondhuru\\Downloads\\DATA_SET"

font_dirs = [os.path.join(data_dir, d) for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
classes = os.listdir(data_dir)
labels = []
for class_name in classes:
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            labels.append(class_name)


In [27]:
start = 0
step = 5000
end = 240000
ranges = []
while start < end:
    ranges.append((start, start+step-1))
    start += step

In [28]:
train_dir = '/content/train'
val_dir = '/content/val'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

i = 0
for font_dir in font_dirs:
    font_name = os.path.basename(font_dir)
    img_file_names = os.listdir(font_dir)

    # Splitting the image file names into train and validation sets
    train_img_file_names, val_img_file_names = train_test_split(img_file_names, test_size=0.2, random_state=42)

    for img_file_name in train_img_file_names:
        src_path = os.path.join(font_dir, img_file_name)
        dst_path = os.path.join(train_dir, font_name, img_file_name)
        os.makedirs(os.path.join(train_dir, font_name), exist_ok=True)
        shutil.copy(src_path, dst_path)

    for img_file_name in val_img_file_names:
        src_path = os.path.join(font_dir, img_file_name)
        dst_path = os.path.join(val_dir, font_name, img_file_name)
        os.makedirs(os.path.join(val_dir, font_name), exist_ok=True)
        shutil.copy(src_path, dst_path)


In [29]:
img_width, img_height = 64, 64
batch_size = 32
epochs = 10

In [30]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
num_classes = len(font_dirs)
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


C:\Users\Midhunsai Pondhuru\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 12)             │         1,548 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 684,748 (2.61 MB)

 Trainable params: 684,748 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

None


In [31]:

classes = ['Georgia', 'Gill sans', 'Helvetica', 'Hombre', 'LCD Mono', 'Lucida Bright', 'Minion', 'Monotype Corsiva', 'Mrs Eaves', 'Myriad', 'Times New Roman', 'Verdana']

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    classes=classes)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    classes=classes)


Found 48008 images belonging to 12 classes.
Found 12010 images belonging to 12 classes.


In [35]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size)

Epoch 1/10


C:\Users\Midhunsai Pondhuru\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 363s 240ms/step - accuracy: 0.7031 - loss: 0.8721 - val_accuracy: 0.9740 - val_loss: 0.0934
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 45us/step - accuracy: 1.0000 - loss: 0.0265 - val_accuracy: 1.0000 - val_loss: 0.0092
Epoch 3/10


c:\Program Files\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 60s 40ms/step - accuracy: 0.9783 - loss: 0.0742 - val_accuracy: 0.9858 - val_loss: 0.0508
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 25us/step - accuracy: 0.9688 - loss: 0.1330 - val_accuracy: 0.9000 - val_loss: 0.2783
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 62s 41ms/step - accuracy: 0.9875 - loss: 0.0429 - val_accuracy: 0.9798 - val_loss: 0.0710
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 10us/step - accuracy: 0.9375 - loss: 0.1104 - val_accuracy: 1.0000 - val_loss: 0.0117
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 69s 46ms/step - accuracy: 0.9906 - loss: 0.0320 - val_accuracy: 0.9909 - val_loss: 0.0355
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 11us/step - accuracy: 1.0000 - loss: 0.0057 - val_accuracy: 1.0000 - val_loss: 2.4692e-04
Epoch 9/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 58s 39ms/step - accuracy: 0.9939 - loss: 0.0206 - val_accuracy: 0.9902 - val_loss: 0.0317
Epoch 10/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 18us/step - accuracy: 1.0000 - loss: 2.4

In [37]:
model.save('my_model.keras')


In [48]:
img_path = "C:\\Users\\Midhunsai Pondhuru\\Downloads\\Image\\Image1.jpg"
img = image.load_img(img_path, target_size=(img_height, img_width))

In [49]:
x = image.img_to_array(img)
x = x / 255.0
x = np.expand_dims(x, axis=0)

# Use the trained model to predict the font
preds = model.predict(x)
pred_class = np.argmax(preds[0])
predicted_font = conv_label(pred_class)
print("Predicted Font:", predicted_font)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted Font: Hombre
